## **Environment setup to run the notebook**
We assume that this notebook will be run on google colab. In order to run this notebook, the following steps have to be performed:


1.   Please make sure your google drive has the following [file](https://drive.google.com/file/d/1QWY78l6kVKk6YE3kT21IVt6N0yNBXIBL/view?usp=sharing) and the [model](https://gitlab.lrz.de/xai-lab-ws2021/edoardo/beyond-simple-word-level-input-relevance-explanations-team-2/-/blob/master/model_weights/grammar_bilstm.p)
2.   Please create the directory '*preprocessing*' and upload [grammar_preprocessing.py](https://gitlab.lrz.de/xai-lab-ws2021/edoardo/beyond-simple-word-level-input-relevance-explanations-team-2/-/blob/master/preprocessing/grammar_preprocessing.py). Please create a directory named '*utils*' and upload the file [utils.py](https://gitlab.lrz.de/xai-lab-ws2021/edoardo/beyond-simple-word-level-input-relevance-explanations-team-2/-/blob/master/utils/util.py).

Once the above setup is done each cell can be run one by one. Please note that your google drive will be mounted to the colab instance. Once the notebook has completed running, we pickle the explainer which will be used for explanations.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/data.zip'

Archive:  /content/drive/MyDrive/data.zip
   creating: data/
  inflating: data/fields             
  inflating: data/train_data         
  inflating: data/vocab              
  inflating: data/test_data          
  inflating: data/label              
  inflating: data/val_data           


In [ ]:
!pip install shap

     |████████████████████████████████| 358kB 17.0MB/s 
  Created wheel for shap: filename=shap-0.38.1-cp36-cp36m-linux_x86_64.whl size=489391 sha256=c7503fdc19f359dbf72e56f9ece63f1a322f1076ae6d65fcd45ef3f73a079cf8
  Stored in directory: /root/.cache/pip/wheels/a8/fb/e4/88012be41842b9be62ae18d82d1b1e880daf8539d1fef1fa00
Successfully built shap


In [ ]:
!pip install pytorch_lightning 

     |████████████████████████████████| 696kB 16.3MB/s 
     |████████████████████████████████| 102kB 14.9MB/s 
     |████████████████████████████████| 276kB 58.8MB/s 
     |████████████████████████████████| 829kB 52.8MB/s 
     |████████████████████████████████| 1.3MB 46.1MB/s 
     |████████████████████████████████| 143kB 50.7MB/s 
     |████████████████████████████████| 296kB 57.1MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=0e5a0548f030fd14116265cd4401d254a1ff64e66bf7e860277446ebfe9a88d2
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=7cfb3a074506e4a067f12d4bc78d2e980766cb40b49ba1273534301ff4e926e7
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl size=316

##**Import the required libraries**


In [ ]:
import shap
from preprocessing.grammar_preprocessing import custom_tokenizer,extract_phrases
import torch
import numpy as np
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from utils import util
import dill
import torchtext
import nltk

##**Load the model**
Here we load the already trained model. We then create the explainers on this model.

In [ ]:
model = util.load_model('/content/drive/MyDrive/biLstm_85.9.p')
model.to(device)
model.eval()

LSTMModel(
  (embedding): Embedding(200002, 100, padding_idx=1)
  (lstm): LSTM(100, 7, num_layers=2, dropout=0.1319029367577066, bidirectional=True)
  (fc): Linear(in_features=14, out_features=1, bias=True)
  (dropout): Dropout(p=0.1319029367577066, inplace=False)
  (criterion): BCEWithLogitsLoss()
)

## **Load the data**
The Kernelexplainer in the shap library nees background data. Hence we load the previously saved data.

In [ ]:
with open("/content/data/vocab","rb")as f:
     textField=dill.load(f)
with open("/content/data/train_data","rb")as f:
     train_data=dill.load(f)
with open("/content/data/val_data","rb")as f:
     val_data=dill.load(f)
with open("/content/data/fields","rb")as f:
     fields=dill.load(f)
train_dataloaders = torchtext.data.BucketIterator(torchtext.data.Dataset(train_data,fields),
                                                                             sort_key = lambda x: len(x.review), batch_size=64,
                                                                             device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

**Define a function that can predict the output of the model** <br/>
We define a predict(), that can take a sentence as an input and then predict the sentiment. The same function will be used by the KernelExplainer while generating the explanations.

In [ ]:
def predict(text):
  with torch.no_grad():
    if isinstance(text,str):
      tokenized = custom_tokenizer(text)
      print(tokenized)
      indexed = [textField.vocab.stoi[t] for t in tokenized]
      tensor = torch.from_numpy(np.array(indexed)).unsqueeze(1).to(device)
    else:
      text = text.T
      indexed = text
      tensor = torch.from_numpy(np.array(indexed)).to(device)

    prediction_logits = model(tensor)
    preds = -(prediction_logits)
    return preds.squeeze(1).cpu().numpy()

In [ ]:
sample = next(iter(train_dataloaders))
background = sample.review
background_numpy = background.cpu().numpy()

## **Generate the explainer for the model**
Here we use the KernelExplainer to generate the explainer for our model. The explainer is then saved for future usage


In [ ]:
explainer = shap.KernelExplainer(predict,background_numpy)

Using 369 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


In [ ]:
with open("/content/background_data","wb")as f:
     dill.dump(background_numpy,f)

In [ ]:
with open("/content/explainer","wb")as f:
     dill.dump(explainer,f)

**Shap values for the background data is generated**

In [ ]:
shap_values = explainer.shap_values(background_numpy,nsamples=150)

In [ ]:
with open("/content/shap_values","wb")as f:
     dill.dump(shap_values,f)